In [ ]:
import pandas as pd
import sqlite3
from sklearn.impute import KNNImputer
import numpy as np
import tqdm
from collections import Counter
import tensorflow as tf
import xgboost as xgb
import math
import statistics
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.feature_selection import SelectKBest,chi2,mutual_info_classif
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

# Connect to SQL DB

In [ ]:
sql_path = "/Users/ariel-pc/Documents/שנה ג/SemesterB/Into to final project/Assignment3/database.sqlite" # PATH to the DB
con = sqlite3.connect(sql_path)

# Queries

In [ ]:
teams_query = """
select t.team_api_id team_id, t.team_long_name team, atr.date, atr.buildUpPlaySpeed,
       atr.buildUpPlayDribbling, atr.buildUpPlayDribblingClass, atr.buildUpPlayPassing, 
       atr.buildUpPlayPositioningClass, atr.chanceCreationPassing,  atr.chanceCreationCrossing,
        atr.chanceCreationShooting, 
       atr.chanceCreationPositioningClass, atr.defencePressure, atr.defenceAggression,
       atr.defenceTeamWidth,  atr.defenceDefenderLineClass
from team t join team_attributes atr on t.team_api_id = atr.team_api_id
"""

players_query = """
select pl.player_api_id player_id, pl.player_name player, pl.birthday, pl.height, pl.weight, pla.date, pla.overall_rating, 
       pla.potential, pla.preferred_foot, pla.attacking_work_rate, pla.defensive_work_rate, pla.crossing, pla.finishing,
       pla.heading_accuracy, pla.short_passing, pla.volleys, pla.dribbling, pla.curve, pla.free_kick_accuracy,
       pla.long_passing, pla.ball_control, pla.acceleration, pla.sprint_speed, pla.agility, pla.reactions, pla.balance,
       pla.shot_power, pla.jumping, pla.stamina, pla.strength, pla.long_shots, pla.aggression, pla.interceptions,
       pla.positioning, pla.vision, pla.penalties, pla.marking, pla.standing_tackle, pla.sliding_tackle, pla.gk_diving,
       pla.gk_handling, pla.gk_kicking, pla.gk_positioning, pla.gk_reflexes
from player pl join player_attributes pla on pl.player_api_id = pla.player_api_id
"""

match_query = """
select m.match_api_id match_id,
c.name country, l.name league, m.season, m.stage, m.date, home.team_long_name home_team,
       away.team_long_name away_team, m.home_team_goal home_goal, m.away_team_goal away_goal, m.goal
from match m, country c, league l, team home, team away
where m.country_id = c.id and m.league_id = l.id and
      home.team_api_id = m.home_team_api_id and away.team_api_id = m.away_team_api_id
"""

# Create the DataFrames

In [ ]:
teams = pd.read_sql_query(teams_query, con)
match = pd.read_sql_query(match_query, con)
players = pd.read_sql_query(players_query, con)
con.close()

# Preprocess the teams DataFrame

In [ ]:
teams_cp = teams.copy()
teams_cp.info()
teams.defenceDefenderLineClass.unique()

# Dummies for positioning class

In [ ]:
teams_w_dummies = pd.get_dummies(teams, columns=['chanceCreationPositioningClass'
                                                 ,'buildUpPlayPositioningClass',
                                                 'defenceDefenderLineClass'])

# Rearrange Team DataFrame for merge

In [ ]:
teams_w_dummies.date.unique()
conditions = [(teams_w_dummies['date'] == '2010-02-22 00:00:00'), 
              (teams_w_dummies['date'] == '2011-02-22 00:00:00'), 
              (teams_w_dummies['date'] == '2012-02-22 00:00:00'), 
              (teams_w_dummies['date'] == '2013-09-20 00:00:00'), 
              (teams_w_dummies['date'] == '2014-09-19 00:00:00'), 
              (teams_w_dummies['date'] == '2015-09-10 00:00:00'), 
             ]
choices = [
            '2009/2010',
            '2010/2011',
            '2011/2012',
            '2012/2013',
            '2013/2014',
            '2014/2015']
teams_w_dummies['season'] = np.select(conditions, choices)

# Feature fill null attributes

In [ ]:
group_by_mean =  teams_w_dummies.groupby('buildUpPlayDribblingClass')['buildUpPlayDribbling'].mean()
break_points = [group_by_mean[0], group_by_mean[2], group_by_mean[1]]
buildUpPlayDribbling_list = []
for i, _class in enumerate(teams_w_dummies.buildUpPlayDribblingClass):
    try:
        if not np.isnan(teams_w_dummies.buildUpPlayDribbling[i]):
            buildUpPlayDribbling_list.append(teams_w_dummies.buildUpPlayDribbling[i])
        elif _class == 'Little' and np.isnan(teams_w_dummies.buildUpPlayDribbling[i]):
            buildUpPlayDribbling_list.append(break_points[0])
        elif _class == 'Normal' and np.isnan(teams_w_dummies.buildUpPlayDribbling[i]):
            buildUpPlayDribbling_list.append(break_points[1])
        elif _class == 'Lots' and np.isnan(teams_w_dummies.buildUpPlayDribbling[i]):
            buildUpPlayDribbling_list.append(break_points[2])
    except Exception as e:
        print(e)
        continue

teams_w_dummies['buildUpPlayDribbling'] = buildUpPlayDribbling_list
teams_w_dummies = teams_w_dummies.drop_duplicates()
del teams_w_dummies['buildUpPlayDribblingClass']

In [ ]:
teams_w_dummies.date = pd.to_datetime(teams_w_dummies.date)

In [ ]:
teams_df = teams_w_dummies[['team_id', 'team','season', 'date', 'buildUpPlaySpeed', 'buildUpPlayDribbling',
       'buildUpPlayPassing', 'chanceCreationPassing', 'chanceCreationCrossing',
       'chanceCreationShooting', 'defencePressure', 'defenceAggression',
       'defenceTeamWidth', 'chanceCreationPositioningClass_Free Form',
       'chanceCreationPositioningClass_Organised',
       'buildUpPlayPositioningClass_Free Form',
       'buildUpPlayPositioningClass_Organised',
       'defenceDefenderLineClass_Cover',
       'defenceDefenderLineClass_Offside Trap']]

In [ ]:
season_list = set(['2008/2009', '2009/2010', '2010/2011', '2011/2012', 
                   '2012/2013','2013/2014', '2014/2015', '2015/2016'])

# Calculate team average score

In [ ]:
teams_df['team_avg_score'] = teams_df.loc[:,'buildUpPlaySpeed':'defenceTeamWidth'].mean(axis=1)

# Match Table

In [ ]:
match_copy = match.copy()
match_copy.date = pd.to_datetime(match_copy.date)
match_copy.info()

# get victories and losses for each team

In [ ]:
points_df = match_copy.loc[:, ['league', 'season', 'home_team', 'away_team','home_goal', 'away_goal']]                                     

# Masks for lambda

In [ ]:
home_win = points_df.home_goal > points_df.away_goal
away_win = points_df.home_goal < points_df.away_goal
home_lose = points_df.home_goal < points_df.away_goal
tie = points_df.home_goal == points_df.away_goal

# Tag 1 as win, 0 as lose, -1 as tie

In [ ]:
points_df['tag'] = np.select([home_win, tie], [1, 0], 2) # Multi label Classification
# points_df['tag'] = np.select([home_win, tie], [1, 0], 0) # Binary Classification

points_df['home_win'] = np.select([home_win], [1], 0)
points_df['away_win'] = np.select([away_win], [1], 0)
points_df['points_in_match'] = np.select([home_win,tie],[3,1],0)

In [ ]:
match_copy['home_win'] = points_df['home_win']
match_copy['away_win'] = points_df['away_win']
match_copy['points_in_match'] = points_df['points_in_match']
match_copy['tag'] = points_df['tag']

# Calculate cumulative away and home wins till now

In [ ]:
match_copy['cum_home_wins'] = match_copy.groupby(['home_team'])['home_win'].cumsum().sub(match_copy.home_win)
match_copy['cum_away_wins'] = match_copy.groupby(['away_team'])['away_win'].cumsum().sub(match_copy.away_win)

In [ ]:
match_copy['ones'] = 1
match_copy['num_games_home'] = match_copy.groupby(['home_team'])['ones'].cumsum()
match_copy['num_games_away'] = match_copy.groupby(['away_team'])['ones'].cumsum()

# Calculate number of games in season

In [ ]:
temp_season = match_copy[['date','season', 'home_team', 'ones']].set_index(['season','home_team','ones'])
temp_season = temp_season.stack().reset_index(name='both')
temp_season['new'] = temp_season.groupby(['season','home_team'])['ones'].cumsum()
match_copy['num_games_home_season'] = temp_season['new']

In [ ]:
temp_season = match_copy[['date','season', 'away_team', 'ones']].set_index(['season','away_team','ones'])
temp_season = temp_season.stack().reset_index(name='both')
temp_season['new'] = temp_season.groupby(['season','away_team'])['ones'].cumsum()
match_copy['num_games_away_season'] = temp_season['new']

# Calculate cumulative away and home wins in season

In [ ]:
temp_season = match_copy[['date','season', 'home_team', 'home_win']].set_index(['season','home_team','home_win'])
temp_season = temp_season.stack().reset_index(name='both')
temp_season['new'] = temp_season.groupby(['season','home_team'])['home_win'].cumsum().sub(temp_season.home_win)
match_copy['cum_home_win_season'] = temp_season['new']

In [ ]:
temp_season = match_copy[['date','season', 'away_team', 'away_win']].set_index(['season','away_team','away_win'])
temp_season = temp_season.stack().reset_index(name='both')
temp_season['new'] = temp_season.groupby(['season','away_team'])['away_win'].cumsum().sub(temp_season.away_win)
match_copy['cum_away_win_season'] = temp_season['new']

In [ ]:
match_df = match_copy.copy()

# Calculate cumulative home and away goals in season

In [ ]:
temp_season = match_df[['season', 'date', 'home_team', 'home_goal']].set_index(['season','home_team','home_goal'])
temp_season = temp_season.stack().reset_index(name='both')
temp_season['new'] = temp_season.groupby(['season','home_team'])['home_goal'].cumsum().sub(temp_season.home_goal)
match_df['cum_home_goal_season'] = temp_season['new']

In [ ]:
temp_season = match_df[['season', 'date', 'away_team', 'away_goal']].set_index(['season','away_team','away_goal'])
temp_season = temp_season.stack().reset_index(name='both')
temp_season['new'] = temp_season.groupby(['season','away_team'])['away_goal'].cumsum().sub(temp_season.away_goal)
match_df['cum_away_goal_season'] = temp_season['new']

In [ ]:
match_df_copy = match_df.copy()

# Previous meetings 

In [ ]:
temp_season = match_df_copy[['date', 'home_team','away_team','home_win']].set_index(['date','home_team','away_team'])
temp_season = temp_season.stack().reset_index(name='both')
temp_season['new'] = temp_season.groupby(['home_team','away_team'])['both'].cumsum().sub(temp_season.both)
match_df_copy['cum_prev_meeting_home'] = temp_season['new']

In [ ]:
temp_season = match_df_copy[['date', 'home_team','away_team','away_win']].set_index(['date','home_team','away_team'])
temp_season = temp_season.stack().reset_index(name='both')
temp_season['new'] = temp_season.groupby(['home_team','away_team'])['both'].cumsum().sub(temp_season.both)
match_df_copy['cum_prev_meeting_away'] = temp_season['new']

# Calculate Proportions

In [ ]:
match_df_copy['total_h_wins_games_portion'] = match_df_copy['cum_home_wins'] / match_df_copy['num_games_home']
match_df_copy['total_a_wins_games_portion'] = match_df_copy['cum_away_wins'] / match_df_copy['num_games_away']

In [ ]:
match_df_copy['total_h_wins_portions_season'] = match_df_copy['cum_home_win_season'] / match_df_copy['num_games_home_season']
match_df_copy['total_a_wins_portions_season'] = match_df_copy['cum_away_win_season'] / match_df_copy['num_games_away_season']

In [ ]:
match_df_copy['total_h_goals_portion'] = match_df_copy['cum_home_goal_season'] / match_df_copy['num_games_home_season']
match_df_copy['total_a_goals_portion'] = match_df_copy['cum_away_goal_season'] / match_df_copy['num_games_away_season']

In [ ]:
match_df_copy['sub_win_portion_teams'] = match_df_copy['cum_home_goal_season'] - match_df_copy['cum_away_goal_season']

In [ ]:
match_df_copy['win_portion'] = match_df_copy['cum_home_wins'] / match_df_copy['cum_away_wins']
match_df_copy['win_portion_season'] = match_df_copy['cum_home_win_season'] / match_df_copy['cum_away_win_season']
match_df_copy['goal_portion'] = match_df_copy['cum_home_goal_season'] / match_df_copy['cum_away_goal_season']

In [ ]:
match_df = match_df_copy[['match_id', 'country', 'league', 'season', 'stage', 'date', 'home_team',
       'away_team', 'home_goal', 'away_goal', 'home_win', 'away_win',
       'points_in_match',  'cum_home_wins', 'cum_away_wins', 'cum_home_win_season', 'cum_away_win_season',
       'cum_home_goal_season', 'cum_away_goal_season', 'cum_prev_meeting_home',
       'cum_prev_meeting_away', 'total_h_wins_games_portion',
       'total_a_wins_games_portion', 'total_h_wins_portions_season',
       'total_a_wins_portions_season', 'total_h_goals_portion',
       'total_a_goals_portion', 'sub_win_portion_teams','tag']]
temp_match_df = match_df.copy()
temp_match_df.isnull().sum()

# Merge Matches and Teams table

In [ ]:
season_list = ['2008/2009', '2009/2010', '2010/2011', '2011/2012', 
                   '2012/2013','2013/2014', '2014/2015', '2015/2016']
idx_dict = {}
for index in range(0,len(season_list)):
    idx_dict[season_list[index]] = index

temp_match_df = match_df.copy()
temp_match_df.info()

# Fix teams dataframe with missing season values

In [ ]:
season_set = set(['2008/2009', '2009/2010', '2010/2011', '2011/2012', 
                   '2012/2013','2013/2014', '2014/2015', '2015/2016'])

In [ ]:
def get_seasons_to_add(season):
    for i in range(0, season):
        yield season_list[i]
def find_seasons(df):
    temp_dict = df.set_index('season').T.to_dict()
    new_df = {}
    keys_set = sorted(temp_dict.keys(), reverse=False)
    missing_seasons = sorted(season_set - set(temp_dict.keys()), reverse=False)
    first_index = idx_dict[keys_set[0]]
    # Handle '2008/2009' -> key_set[0]
    for test in get_seasons_to_add(first_index):
        temp_dict[test] = temp_dict[keys_set[0]]
        
    # Handle rest of missing values
    for i in range(1, len(missing_seasons)):
        temp_dict[missing_seasons[i]] = temp_dict[missing_seasons[i-1]]
    return temp_dict

In [ ]:
teams_list = teams_df.team.unique()
new_team_df = pd.DataFrame(columns=teams_df.columns)
for team_name in tqdm.tqdm(teams_list):
    df_to_concat = find_seasons(teams_df[teams_df['team'] == team_name])
    df_to_concat = pd.DataFrame(df_to_concat).T.reset_index()
    df_to_concat = df_to_concat.rename(columns = {'index':'season'})
    new_team_df = pd.concat([new_team_df, df_to_concat], ignore_index=True)

In [ ]:
new_team_df = new_team_df.sort_values(['team','season'])

# Merge Match table with Team table

In [ ]:
england_league = temp_match_df[match_df["league"]=="England Premier League"]
cols_to_add = list(new_team_df.columns[5:])

In [ ]:
def fit_cols_home(df1, df2,string):
    for col in cols_to_add:
        df1[col+'_'+string] = df2[col]
    return df1

In [ ]:
cols_to_add = ['buildUpPlaySpeed',
       'buildUpPlayDribbling', 'buildUpPlayPassing', 'chanceCreationPassing',
       'chanceCreationCrossing', 'chanceCreationShooting', 'defencePressure',
       'defenceAggression', 'defenceTeamWidth',
       'chanceCreationPositioningClass_Free_Form',
       'chanceCreationPositioningClass_Organised',
       'buildUpPlayPositioningClass_Free_Form',
       'buildUpPlayPositioningClass_Organised',
       'defenceDefenderLineClass_Cover',
       'defenceDefenderLineClass_Offside_Trap','team_avg_score']
new_list_to_add_home = [col + '_home' for col in cols_to_add]
home_dict = dict(zip(cols_to_add,new_list_to_add_home))

new_list_to_add_away = [col + '_away' for col in cols_to_add]
away_dict = dict(zip(cols_to_add, new_list_to_add_away))

In [ ]:
df = temp_match_df.merge(new_team_df, left_on=['home_team','season'], right_on=['team','season'],suffixes=('_home','_home'))
df = df.drop(columns =['team_id', 'team','date_home'])
df = df.rename(columns = home_dict)

df = df.merge(new_team_df, left_on=['away_team','season'], right_on=['team','season'],suffixes=('_away','_away'))
df = df.rename(columns = away_dict)
df = df.drop(columns=['chanceCreationPositioningClass_Free Form_away',
                      'defenceDefenderLineClass_Offside Trap_away',
                     'buildUpPlayPositioningClass_Free Form_away'])

# Remove unneseccary columns

In [ ]:
df = df.drop(columns=['country','league','stage','home_team','away_team','home_goal','away_goal','home_win','away_win','points_in_match'])

In [ ]:
df.columns
col_list = ['match_id','season', 'cum_home_wins', 'cum_away_wins', 'cum_home_win_season',
       'cum_away_win_season', 'cum_home_goal_season', 'cum_away_goal_season',
       'cum_prev_meeting_home', 'cum_prev_meeting_away',
       'total_h_wins_games_portion', 'total_a_wins_games_portion',
       'total_h_wins_portions_season', 'total_a_wins_portions_season',
       'total_h_goals_portion', 'total_a_goals_portion',
       'sub_win_portion_teams',  'team_avg_score_home','buildUpPlaySpeed_home',
       'buildUpPlayDribbling_home', 'buildUpPlayPassing_home',
       'chanceCreationPassing_home', 'chanceCreationCrossing_home',
       'chanceCreationShooting_home', 'defencePressure_home',
       'defenceAggression_home', 'defenceTeamWidth_home',
       'chanceCreationPositioningClass_Organised_home',
       'buildUpPlayPositioningClass_Organised_home',
       'defenceDefenderLineClass_Cover_home', 'team_id', 'team', 'date','team_avg_score_away',
       'buildUpPlaySpeed_away', 'buildUpPlayDribbling_away',
       'buildUpPlayPassing_away', 'chanceCreationPassing_away',
       'chanceCreationCrossing_away', 'chanceCreationShooting_away',
       'defencePressure_away', 'defenceAggression_away',
       'defenceTeamWidth_away',
       'chanceCreationPositioningClass_Organised_away',
       'buildUpPlayPositioningClass_Organised_away',
       'defenceDefenderLineClass_Cover_away','tag']

df = df[col_list]

# Preprocces and prepare integration of players features to final table

In [ ]:
con = sqlite3.connect(sql_path)
match_query = """
select m.match_api_id match_id,
m.home_player_1 home_pla_1, m.home_player_2 home_pla_2, m.home_player_3 home_pla_3, m.home_player_4 home_pla_4, m.home_player_5 home_pla_5,
m.home_player_6 home_pla_6, m.home_player_7 home_pla_7, m.home_player_8 home_pla_8, m.home_player_9 home_pla_9, m.home_player_10 home_pla_10,m.home_player_11 home_pla_11,  
m.away_player_1 away_pla_1, m.away_player_2 away_pla_2, m.away_player_3 away_pla_3, m.away_player_4 away_pla_4, m.away_player_5 away_pla_5,
m.away_player_6 away_pla_6, m.away_player_7 away_pla_7, m.away_player_8 away_pla_8, m.away_player_9 away_pla_9, m.away_player_10 away_pla_10,m.away_player_11 away_pla_11
from match m
"""
match_temp_naor = pd.read_sql_query(match_query, con)
con.close()

# Clean Matchs with NaN player id

In [ ]:
match_temp_naor=match_temp_naor.dropna(how='any',axis=0)
match_temp_naor.info()

# Preprocess Players DataFrame

In [ ]:
players = players.drop_duplicates()
players_cp=players.copy()
sum_of_NaN_values=players.isnull().sum()
is_NaN = players. isnull()
row_has_NaN = is_NaN.any(axis=1)
# row_has_NaN
rows_with_NaN = players[row_has_NaN]

# let's brake down the problem to suspicious Nan accurances: 

# group 835:

first, lets investigate all the attributes that are with 835 Nan values:

In [ ]:
#lets try and take the cleaning aprouch by the repeating numbers in the rows which contain none
#maybe those are certain players that needs to be removed(almost empty records).
attr_with_NaN_835=['player_id', 'player']
for row in sum_of_NaN_values.iteritems():
    if row[1]==835:
        attr_with_NaN_835.append(row[0])
# print(attr_with_NaN_835)
players_only_835_attr =  rows_with_NaN[attr_with_NaN_835]
rows_835_with_NaN=players_only_835_attr[(players_only_835_attr.isnull()).any(axis=1)]
print(len(rows_835_with_NaN))
rows_835_with_NaN.head(100)

as suspected, there is correlation between all these NaN values, they all are part of empty
records of players(lack of data).
we can deal with this group of NaN values by removing the records of the players involved.

In [ ]:
#get player_id for each record we want to remove
player_indexs_to_remove=rows_835_with_NaN.index
players_cp.drop(player_indexs_to_remove, inplace=True)
players_cp.info()

as we can see, there are 835 less records in the players data frame.
now, lets take a look at the NaN values again.

In [ ]:
sum_of_NaN_values=players_cp.isnull().sum()
print(sum_of_NaN_values)

# group 1877:

again, lets investigate all the attributes that are with 1877 Nan values:

In [ ]:
#update the rows with Nan left to handle
rows_with_NaN= players_cp[players_cp.isnull().any(axis=1)]
attr_with_NaN_1877=['player_id', 'player']

for row in sum_of_NaN_values.iteritems():
    if row[1]==1877:
        attr_with_NaN_1877.append(row[0])

players_only_1877_attr =  rows_with_NaN[attr_with_NaN_1877]
# players_only_1877_attr
rows_1877_with_NaN=players_only_1877_attr[(players_only_1877_attr.isnull()).any(axis=1)]
print(len(rows_1877_with_NaN))
rows_1877_with_NaN.head(100)

looks as if its the same case like group 835  only that this time it's player related. 
can be understood from the fact that the 835 group contained distinct records of players whilst here we have numerous records of some of the players.
either way it will be dealt with likewise.

In [ ]:
#get indexs for each record we want to remove
player_indexs_to_remove=rows_1877_with_NaN.index
players_cp.drop(player_indexs_to_remove, inplace=True)
sum_of_NaN_values=players_cp.isnull().sum()
print(sum_of_NaN_values)

# group attacking_work_rate:

first, lets take a look at the values of the problematic attribute, since its of object type.

In [ ]:
players_cp.attacking_work_rate.value_counts()

In [ ]:
players_cp.attacking_work_rate.hist(bins=20)

looking at the distribution of this attribute we can assume that we can take the values: "None", "le", "norm", "stoc" and "y" and classify them as "medium".that classification should not damage the contribution of the attribute's values since the majority of the players are classified as "medium".
now, we will make the attribute numeric, classifying:
"low" as 1
"medium" as 2
and
"high" as 3.
the context is preserved because 1, 2 and 3 have the same mathematical relation as low, medium and high.

In [ ]:
attacking_work_rate_list = []
for i, _class in enumerate(players_cp.attacking_work_rate):
    try:
#         print(type(_class))
        if _class == "low":
            attacking_work_rate_list.append(1)
        elif _class == "high":
            attacking_work_rate_list.append(3)
        else:
            attacking_work_rate_list.append(2)
    except Exception as e:
        print(e)
        continue

players_cp['attacking_work_rate'] = attacking_work_rate_list
players_cp = players_cp.drop_duplicates()
players_cp.attacking_work_rate.value_counts()
players_cp.attacking_work_rate.hist(bins=20)

In [ ]:
sum_of_NaN_values=players_cp.isnull().sum()
print(sum_of_NaN_values)

after proccesing the table to the point which there are no NaN values, it is needed to look at the non numeric value types and consider further actions in the preproccess.

In [ ]:
for attribute_column in players_cp:
    if players_cp.dtypes[attribute_column] == np.object:
        print(players_cp[attribute_column].value_counts())

# group defensive_work_rate:

In [ ]:
players_cp.defensive_work_rate.value_counts()

In [ ]:
players_cp.defensive_work_rate.hist(bins=20)

looking at the distribution of this attribute we can assume that we can take the values: "ean", "es", "ormal", "_0", "tocky" and "0" to "9" and classify them as "medium".that classification should not damage the contribution of the attribute's values since the majority of the players are classified as "medium".
now, we will make the attribute numeric, classifying:
"low" as 1
"medium" as 2
and
"high" as 3.
the context is preserved because 1, 2 and 3 have the same mathematical relation as low, medium and high.

In [ ]:
defensive_work_rate_list = []
for i, _class in enumerate(players_cp.defensive_work_rate):
    try:
#         print(type(_class))
        if _class == "low":
            defensive_work_rate_list.append(1)
        elif _class == "high":
            defensive_work_rate_list.append(3)
        else:
            defensive_work_rate_list.append(2)
    except Exception as e:
        print(e)
        continue

players_cp['defensive_work_rate'] = defensive_work_rate_list
players_cp = players_cp.drop_duplicates()
players_cp.defensive_work_rate.value_counts()
players_cp.defensive_work_rate.hist(bins=20)

# preferred_foot  drop: 

In [ ]:
players_cp.drop('preferred_foot',axis=1,inplace=True)

In [ ]:
for attribute_column in players_cp:
    if players_cp.dtypes[attribute_column] == np.object:
        print(players_cp[attribute_column].value_counts())

# creating age feature:
we want to add "age" feature to the player dataset after Considerat the contribution of the attribute's values.

In [ ]:
players_cp['date_year'] = pd.to_datetime(players_cp['date'], format='%Y-%m-%d %H:%M:%S')
players_cp['birthday_year'] = pd.to_datetime(players_cp['birthday'], format='%Y-%m-%d %H:%M:%S')
print (players_cp['date_year'])
print (players_cp['birthday_year'])
players_cp['date_year'] = players_cp.date_year.apply(lambda x: x.year)
players_cp['birthday_year'] = players_cp.birthday_year.apply(lambda x: x.year)
print (players_cp['date_year'])
print (players_cp['birthday_year'])

In [ ]:
players_cp['age']= players_cp['date_year'] - players_cp['birthday_year']
print (players_cp[['date_year', 'birthday_year', 'age' ]])
print (players_cp.dtypes)

# birthday ,birthday_year and date_year drop:
Now after we extract the age featue we will drop the birthday feature (in addition to birthday_year and date_year) :

In [ ]:
players_cp.drop('birthday',axis=1,inplace=True)
players_cp.drop('date_year',axis=1,inplace=True)
players_cp.drop('birthday_year',axis=1,inplace=True)
print (players_cp.dtypes)
print (players_cp.columns)

# player name drop:
as we can see we left with 2 features from type object: date and player.
we need the date to merge tables, but the player name feature gives no contribution so we have decided to drop it:

In [ ]:
players_cp.drop('player',axis=1,inplace=True)
players_cp[['player_id','date','overall_rating']].head(20)

# players feature extraction + selection

# BMI + Age: 

In [ ]:

avg_prime_age=26
lbs_to_kgs_factor=0.45359237
BMI_age_list=[]
for idx,p in players_cp[['height','weight','age']].iterrows():
    new_att = ((p.weight*lbs_to_kgs_factor)/(math.pow((p.height/100),2)))*((avg_prime_age/p.age)*0.5)
    BMI_age_list.append(new_att)
    
players_cp['BMI_age_corelay']=BMI_age_list
players_cp.drop(['height','weight'],axis='columns', inplace=True)

# Unitary attributes combined to 1 feature

In [ ]:
# players_cp.columns
list_of_relevant_attr=['crossing', 'finishing',
       'heading_accuracy', 'short_passing', 'volleys', 'dribbling', 'curve',
       'free_kick_accuracy', 'long_passing', 'ball_control', 'acceleration',
       'sprint_speed', 'agility', 'reactions', 'balance', 'shot_power',
       'jumping', 'stamina', 'strength', 'long_shots', 'aggression',
       'interceptions', 'positioning', 'vision', 'penalties', 'marking',
       'standing_tackle', 'sliding_tackle', 'gk_diving', 'gk_handling',
       'gk_kicking', 'gk_positioning', 'gk_reflexes']

list_of_comb_attr=[]

num_of_attr = len(list_of_relevant_attr)
for idx , p in players_cp[list_of_relevant_attr].iterrows():
    sum_of_attr=0
    rec_avg=0
    for attr in p.values:
        sum_of_attr+=attr
    rec_avg=(sum_of_attr/num_of_attr)
    list_of_comb_attr.append(rec_avg)

players_cp['avg_attr_rating']=list_of_comb_attr
players_cp.drop(list_of_relevant_attr,axis='columns', inplace=True)


# fit player statistics to the season format

In [ ]:
players_cp['date']=pd.to_datetime(players_cp['date'], format='%Y-%m-%d %H:%M:%S')
players_cp['date']=pd.DatetimeIndex(players_cp['date']).year
players_cp.drop_duplicates(subset=['player_id','date'], inplace=True)
season_to_year_dict = { '2006':'2006/2007','2007':'2007/2008','2008':'2008/2009', '2009':'2009/2010', '2010':'2010/2011', '2011':'2011/2012', 
                   '2012':'2012/2013','2013':'2013/2014', '2014':'2014/2015','2015':'2015/2016','2016':'2016/2017'}
season_list = []
for index, year in players_cp.date.iteritems():
    season_list.append(season_to_year_dict[str(year)])
players_cp['season']=season_list
del players_cp['date']

# merge temp match table with players proccessed table

In [ ]:
p_num_list=['home_pla_1', 'home_pla_2', 'home_pla_3', 'home_pla_4','home_pla_5',
 'home_pla_6',  'home_pla_7',  'home_pla_8', 'home_pla_9',  'home_pla_10', 'home_pla_11',  
 'away_pla_1',  'away_pla_2',  'away_pla_3',  'away_pla_4',  'away_pla_5',
 'away_pla_6',  'away_pla_7',  'away_pla_8', 'away_pla_9',  'away_pla_10', 'away_pla_11']
p_num_extend=[]
for p_num in p_num_list:
    for col_name in players_cp.columns[1:]:
        if col_name != 'season':
            p_num_extend.append(p_num+'_'+col_name)

# Fill new players values with NaN

In [ ]:
NaN_idle_fil_list=[np.NaN]*len(match_temp_naor['match_id'])
for c in p_num_extend:
    match_temp_naor[c]=NaN_idle_fil_list
df=pd.merge(df,match_temp_naor, on='match_id')

# Fill the values of the Match with the Players values

In [ ]:
tmp_season=players_cp['season']
del players_cp['season']
players_cp['season']=tmp_season

In [ ]:
def season_is_ok(season,df):
    tmp = df.loc[df['season']==season]
    if tmp.empty:
        return False
    return True

In [ ]:
def change_to_existing_season(season_missing,df):
    season_to_year_list = ['2006/2007','2007/2008','2008/2009', '2009/2010','2010/2011', '2011/2012', 
                   '2012/2013','2013/2014','2014/2015','2015/2016','2016/2017']
    
    season_index = season_to_year_list.index(season_missing)
    while season_index>0:
        season_index-=1
        if season_is_ok(season_to_year_list[season_index],df):
            return season_to_year_list[season_index]
    for i in range(0,len(season_to_year_list)):
        if season_is_ok(season_to_year_list[i],df):
            return season_to_year_list[i]
    return False

In [ ]:
for idx,match_tmp in df.iterrows():
    for p_num,p_x in enumerate(p_num_list):
        from_idx=69+(7*p_num)
        to_idx= from_idx+7
        try:
            temp_var = players_cp.loc[(players_cp['season']==match_tmp['season'])&(players_cp['player_id']==match_tmp[p_x])]
            if(temp_var.empty):
                good_season=change_to_existing_season(match_tmp['season'], players_cp.loc[players_cp['player_id']==match_tmp[p_x]])
                if good_season==False :
                    temp_var = [-1]*7
                else:
                    temp_var = players_cp.loc[(players_cp['season']==good_season)&(players_cp['player_id']==match_tmp[p_x])].iloc[0,1:8]
            else:
                temp_var = temp_var.iloc[0,1:8]
            data_to_add=list(temp_var)
            df.iloc[idx,from_idx:(to_idx)]=data_to_add
            
        except Exception as e:
            raise(e)

# fill NaN values with mean of the positional player of all players

In [ ]:
values={}
for col in p_num_extend:
    values[col] = df[col].mean()
df.fillna(value=values,inplace=True)

In [ ]:
sum_of_NaN_values=df.isnull().sum()

In [ ]:
# df.to_csv('/Users/aspir/Documents/studies/year3/proj_prep/ML/data_afterpreprocess_united.csv')

# Team based feature extraction

In [ ]:
df.columns[50]
print(df.iloc[0,116:182:6])

In [ ]:
def avg_of_positional(l):
    tmp_sfdgh=statistics.mean(l)
    return tmp_sfdgh

# Calculate the player portions by team

In [ ]:

counter = 2
lst_home_overall = []
lst_home = []
lst_away_overall = []
lst_away = []
lst_home_avg_attr_rating = []
lst_away_avg_attr_rating = []
lst_home_def_rate = []
lst_away_def_rate = []
lst_home_att_rate = []
lst_away_att_rate = []

for index,row in tqdm.tqdm(df.iterrows()):
    lst_home.append(statistics.mean(list(row.iloc[49:115:6])))
    lst_home_overall.append(statistics.mean(list(row.iloc[48:114:6])))
    lst_home_avg_attr_rating.append(statistics.mean(list(row.iloc[53:119:6])))
    lst_home_def_rate.append(statistics.mean(list(row.iloc[51:117:6])))
    lst_home_att_rate.append(statistics.mean(list(row.iloc[50:114:6])))
    lst_away.append(statistics.mean(list(row.iloc[115:181:6])))
    lst_away_overall.append(statistics.mean(list(row.iloc[114:180:6])))
    lst_away_avg_attr_rating.append(statistics.mean(list(row.iloc[119:184:6])))
    lst_away_def_rate.append(statistics.mean(list(row.iloc[117:182:6])))
    lst_away_att_rate.append(statistics.mean(list(row.iloc[116:182:6])))

In [ ]:
df['avg_potential_home'] = lst_home
df['avg_potential_away'] = lst_away
df['avg_overall_rating_home'] = lst_home_overall
df['avg_overall_rating_away'] = lst_away_overall
df['avg_attr_rating__home'] = lst_home_avg_attr_rating
df['avg_attr_rating__away'] = lst_away_avg_attr_rating
df['avg_def_rate_home'] = lst_home_def_rate
df['avg_def_rate_away'] = lst_away_def_rate
df['avg_att_rate_home'] = lst_home_att_rate
df['avg_att_rate_away'] = lst_away_att_rate

In [ ]:
# df.to_csv('/Users/aspir/Documents/studies/year3/proj_prep/ML/data_afterpreprocess_united.csv')

# Train the model

In [ ]:
# dataset = pd.read_csv('/Users/aspir/Documents/studies/year3/proj_prep/ML/data_afterpreprocess_united.csv')
dataset = df.copy()

In [ ]:
for i in range(1,12):
    del dataset['away_pla_'+str(i)+'_defensive_work_rate']
    del dataset['home_pla_'+str(i)+'_defensive_work_rate']

    del dataset['away_pla_'+str(i)+'_attacking_work_rate']
    del dataset['home_pla_'+str(i)+'_attacking_work_rate']
    
    del dataset['home_pla_' +str(i)+'_BMI_age_corelay']
    del dataset['away_pla_' +str(i)+'_BMI_age_corelay']

In [ ]:
del dataset['Unnamed: 0']
del dataset['Unnamed: 0.1']
del dataset['match_id']
del dataset['team']
del dataset['date']
del dataset['team_id']
lst_to_del = ['defenceDefenderLineClass_Cover_home',
             'defenceTeamWidth_away',
             'defenceTeamWidth_home',
             'defenceAggression_away',
             'defenceAggression_home',
             'chanceCreationPassing_away',
             'chanceCreationPassing_home',
             'chanceCreationCrossing_away',
             'defenceDefenderLineClass_Cover_away',
             'buildUpPlayDribbling_away',
             'chanceCreationCrossing_home',
             'buildUpPlaySpeed_home',
             'buildUpPlaySpeed_away',
             'buildUpPlayDribbling_home',
             'chanceCreationShooting_away']

# Calculate portions of team statics

In [ ]:
dataset = dataset.drop(columns = lst_to_del)
dataset['team_avg_portion'] = dataset['team_avg_score_home'] / dataset['team_avg_score_away']
dataset['buildUpPlayPassing_portion'] = dataset['buildUpPlayPassing_home'] / dataset['buildUpPlayPassing_away']
dataset['defencePressure_portion'] = dataset['defencePressure_home'] / dataset['defencePressure_away']
dataset['win_portion'] = dataset['cum_home_wins'] / dataset['cum_away_wins']
dataset['win_portion_season'] = dataset['cum_home_win_season'] / dataset['cum_away_win_season']
dataset['goal_portion'] = dataset['cum_home_goal_season'] / dataset['cum_away_goal_season']
dataset = dataset.replace(np.nan,0)
dataset.replace([np.inf, -np.inf], 0, inplace=True)

In [ ]:
temp = dataset['tag']
del dataset['tag']
dataset['tag'] = temp

# Remove unnessecary columns after feature extraction and re arrange columns

In [ ]:
del dataset['team_avg_score_away']
del dataset['team_avg_score_home']
del dataset['team_avg_portion']

# Split to train and test 

In [ ]:
train = dataset[dataset['season'] < '2015/2016']
del train['season']
test = dataset[dataset['season'] >= '2015/2016']
del test['season']
del dataset['season']

In [ ]:
x_train, y_train = train.iloc[:,:-1], train.iloc[:,-1:]
x_test, y_test = test.iloc[:,:-1], test.iloc[:,-1:]

# Define scaler and fit the data

In [ ]:
scaler = MinMaxScaler()
scaled_train = scaler.fit_transform(x_train)
scaled_test = scaler.fit_transform(x_test)

# feature selection

In [ ]:
X_new = SelectKBest(chi2, k=2)
fit = X_new.fit(scaled_train, y_train)

features_scores = []
list_features = list(dataset.columns)
for idx in range(len(X_new.scores_)):
    features_scores.append((list_features[idx],X_new.scores_[idx]))
sorted_features = sorted(features_scores,key=lambda x: x[1],reverse=True)
sorted_features

# XGBoost

# Grid Search for hyperparmeters

In [ ]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }
xgb = xgb.XGBClassifier(learning_rate=0.02, n_estimators=600, objective='multi:softmax',
                    silent=True, nthread=-1)
grid = GridSearchCV(xgb, param_grid=params, scoring='accuracy', 
                                   n_jobs=4,  verbose=3)
grid.fit(scaled_train, y_train)

# Train the model

In [ ]:
param = {
    'colsample_bytree' : 0.5,
    'max_depth': 7,  # the maximum depth of each tree
    'min_child_weight' : 5,
    '_estimators' : 100,
    'subsample' : 0.5,
    'objective': 'multi:softmax',  # error evaluation for multiclass training
#     'objective': 'binary:logistic', # binary classifier
    'num_class': 3
    }  # the number of classes that exist in this datset
num_round = 20  # the number of training iterations
xgbmodel = xgb.XGBClassifier(param)

In [ ]:
xgbmodel.fit(scaled_train, y_train)
xgbmodel.score(scaled_test, y_test)

# Evaluate the model

In [ ]:
y_pred = xgbmodel.predict(scaled_test)
cm = metrics.confusion_matrix(y_test, y_pred, labels = [2,1,0])

# ANN

In [ ]:
# model parameters
input_size = 106
output_size = 3
# output_size = 2 # Binary Classifier
hidden_layer_size = 100
batch_size = 64
max_epochs = 1000

model = tf.keras.Sequential([
    tf.keras.layers.Dense(input_size), 
    tf.keras.layers.Dense(hidden_layer_size, activation='sigmoid'), 
    tf.keras.layers.Dense(hidden_layer_size, activation='sigmoid'), 
    tf.keras.layers.Dense(hidden_layer_size, activation='sigmoid'), 
    tf.keras.layers.Dense(output_size, activation='softmax')])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

# create the model
model.compile(optimizer=optimizer, loss='SparseCategoricalCrossentropy', metrics=['accuracy'])


early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

# train the model
model.fit(scaled_train, 
          np.array(y_train), 
          batch_size=batch_size, 
          epochs=max_epochs,
          callbacks=[early_stopping],
          validation_data=(scaled_test, np.array(y_test)),
          verbose = 2) 
test_loss, test_accuracy = model.evaluate(scaled_test, np.array(y_test))

In [ ]:
y_pred = model.predict_classes(scaled_test)

# Random forest

# Grid Search for hyperparametrs

In [ ]:
rf = RandomForestClassifier()
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,
                               n_iter = 100, cv = 3, verbose=3, random_state=42, n_jobs = -1)

# Train the model

In [ ]:
params = {'n_estimators': 2000,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 10,
 'bootstrap': True}
rf = RandomForestClassifier(n_estimators=2000,min_samples_split=5, min_samples_leaf=1, max_features='sqrt',
                           max_depth=10, bootstrap=True)
rf.fit(x_train, y_train.ravel)

# Evaluate the Model

In [ ]:
y_pred = rf.predict(scaled_test)
print(metrics.classification_report(y_test,y_pred, labels=[0,1,2]))

# CATBoost

# Train the model

In [ ]:
catboost_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    random_strength=0.1,
    depth=8,
    loss_function='MultiClass',
    eval_metric='Accuracy',
    leaf_estimation_method='Newton'
)

In [ ]:
catboost_model.fit(scaled_train, y_train, verbose=True)

# Evaluate the model

In [ ]:
y_pred = catboost_model.predict(scaled_test)
# print("F1 score", metrics.f1_score(y_test,y_pred, average='micro'))
print(metrics.classification_report(y_test,y_pred, labels=[0,1]))